## Extinction, galactic latitude and peak absolute magnitude

### To run this notebook, please [follow the instructions](https://lasair-lsst.readthedocs.io/en/main/core_functions/python-notebooks.html) or else it won`t work.
The instructions are at https://lasair-lsst.readthedocs.io/en/main/core_functions/python-notebooks.html

This notebook computes three Lasair features: extinction E(B-V), galactic latitude, 
and peak extinction-corrected absolute magnitude (PECAM).

In [1]:
import math, json, redshift_distance

### 1. Extinction
#### Get the dustmaps package set up. 
We use this package to compute the extinction from the dustmap of 
[Schlegel, Finkbeiner, and Davis](https://iopscience.iop.org/article/10.1086/305772).
Once the package is installed, the dustmaps themselves must be downloaded with fetch() -- change the
directory name according to your own setup.
Note that the fetch() method only needs to be called once. 

In [2]:
# Uncomment this to install the dustmaps package
#!/usr/bin/pip3 install dustmaps
from dustmaps.config import config

# Set this to where you want the two 64 Mbyte dustmap files
config['data_dir'] = '/Users/rwillia5/Desktop'
import dustmaps.sfd

# Uncomment the following to download the SFD dustmap
# dustmaps.sfd.fetch()

from dustmaps.sfd import SFDQuery
from astropy.coordinates import SkyCoord
sfd = SFDQuery()

/Users/rwillia5/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


#### Corrected colour for extinction
These colour corrections are multiplied by the extinction E(B-V) to get a magnitude correction. 
They arefrom Table 6 of 
[Schlafly and Finkbeiner](https://iopscience.iop.org/article/10.1088/0004-637X/737/2/103) with RV=3.1

In [3]:
# The LSST bands
EXTCOEF   = {'u':4.145, 'g':3.237, 'r':2.273, 'i':1.684, 'z':1.323, 'y':1.088}

# Modify magnitude for extinction as
# mag - ebv*EXTCOEF[band]

#### Extinction from sky position

In [4]:
def computeExtinction(ra, decl):
    c = SkyCoord(ra, decl, unit="deg", frame='icrs')
    ebv = float(sfd(c))
    return ebv

### 2. Magnitude and Flux
#### Transform between magnitudes and nanoJanskies

In [5]:
def mag2flux(mag):
    # flux in nanoJ
    flux =  math.pow(10, (31.4 - mag)/2.5)
    return flux
def flux2mag(flux):
    # flux in nanoJ
    mag = 31.4 - 2.5*math.log10(flux)
    return mag

### 3. Peak extinction-corrected magnitude
From the E(B-V) and lightcurve in flux, we compute magnitude, 
correct them with extinction, then find the brightest point, and the associated band.

In [6]:
def findPeakExtMag(ebv, z, lightcurve):
    peakMag  = 100
    peakBand = None
    for diaSource in lightcurve:
        mjd        = diaSource['midpointMjdTai']
        band       = diaSource['band']
        psfFlux    = diaSource['psfFlux']
        psfFluxErr = diaSource['psfFluxErr']
        if psfFlux > 0:
            mag = flux2mag(psfFlux)
            # extinction correction and k-correction
            correctedMag = mag - ebv*EXTCOEF[band] + 2.5*math.log(1+z)
            if correctedMag < peakMag:
                peakMag = correctedMag
                peakBand = band
    return (peakMag, peakBand)

### 4. Galactic latitude

In [7]:
# https://en.wikipedia.org/wiki/Galactic_coordinate_system
def galacticLat(ra, decl):
    alphaNGP = 192.85948
    deltaNGP =  27.1283
    sdngp = math.sin(math.radians(deltaNGP))
    cdngp = math.cos(math.radians(deltaNGP))
    sde = math.sin(math.radians(decl))
    cde = math.cos(math.radians(decl))
    cra = math.cos(math.radians(ra - alphaNGP))
    glat = math.degrees(math.asin(sdngp*sde + cdngp*cde*cra))
    return glat

### 5. Test run
Use the Lasair API to find some objects with a host galaxy, then compute their features.
Make sure to connect to the right endpoint.
The `cache` argument is the name of a directory where API calls can be cached, so you don't
need to go to the Lasair server for the same query a second time (can be omotted).

In [8]:
#!/usr/bin/pip3 install lasair
from lasair import LasairError, lasair_client as lasair
import settings
endpoint = "https://api.lasair-lsst.lsst.ac.uk/api"
!mkdir cache
L = lasair(settings.API_TOKEN, endpoint=endpoint, cache='cache')

mkdir: cache: File exists


#### Get some working objects with a Sherlock redshift

In [9]:
selected = """ 
  objects.diaObjectId, objects.ra, objects.decl, 
  sherlock_classifications.z, sherlock_classifications.photoz
"""
tables = 'objects,sherlock_classifications'
conditions = """
  classification="SN" AND 
  (sherlock_classifications.z > 0 OR sherlock_classifications.photoz > 0)
"""
results = L.query(selected, tables, conditions, limit = 50)
for row in results:
    objectId = row['diaObjectId']
    ra       = row['ra']
    decl     = row['decl']
    z        = row['z']
    photoz   = row['photoz']
    if not z:
        z = photoz
    object   = L.object(objectId, lasair_added=False, lite=True)

    # compute galactic latitude
    ebv = computeExtinction(ra, decl)
    
    # compute extinction
    galLat = galacticLat(ra, decl)

    # compute peak extinction-corrected apparent magnitude
    (peakMag, peakBand) = findPeakExtMag(ebv, z, object['diaSources'])
    if not peakBand:
        continue

    # combine z and apparent mag to get absolute mag
    # using Ken Smith code from Atlas for distance modulus
    distances = redshift_distance.redshiftToDistance(z)
    distanceModulus = distances['dmod']

    absMag = peakMag - distanceModulus

    print('%s: ra=%.2f decl=%.2f z=%.3f E(B-V)=%.2f peakMax=%.2f peakBand=%s, absMag=%.2f' % 
          (objectId, ra, decl, z, ebv, peakMag, peakBand, absMag))


3516506109982539799: ra=52.84 decl=-26.57 z=0.014 E(B-V)=0.01 peakMax=23.30 peakBand=g, absMag=-10.61
3516506171722694675: ra=51.97 decl=-28.46 z=0.014 E(B-V)=0.01 peakMax=20.97 peakBand=u, absMag=-12.94
3516508308468924426: ra=53.02 decl=-28.29 z=0.005 E(B-V)=0.01 peakMax=19.72 peakBand=u, absMag=-11.94
3516508308468924434: ra=53.09 decl=-28.37 z=0.005 E(B-V)=0.01 peakMax=23.85 peakBand=u, absMag=-7.82
3521874336992460809: ra=51.85 decl=-27.66 z=0.018 E(B-V)=0.01 peakMax=23.36 peakBand=g, absMag=-11.11
3521874336992460811: ra=51.85 decl=-27.62 z=0.018 E(B-V)=0.01 peakMax=22.57 peakBand=g, absMag=-11.89
3521874828229345465: ra=52.01 decl=-29.18 z=0.004 E(B-V)=0.01 peakMax=20.79 peakBand=g, absMag=-10.39
3521874828229345510: ra=51.97 decl=-29.18 z=0.004 E(B-V)=0.01 peakMax=20.16 peakBand=g, absMag=-11.02
3521874828229345584: ra=52.01 decl=-29.13 z=0.004 E(B-V)=0.01 peakMax=20.27 peakBand=g, absMag=-10.91
3521874828229345592: ra=51.95 decl=-29.17 z=0.004 E(B-V)=0.01 peakMax=21.28 peakBan